In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

In [ ]:
# Uploading CSV DataSet
file_path = '/content/balanced_dataset.csv'  # Update with the actual filename if necessary
df = pd.read_csv(file_path)

# Checking if data has been loaded accurately by loading first few rows
df.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
# Checking for missing values
df.isnull().sum()

,0
Destination Port,0
Flow Duration,0
Total Fwd Packets,0
Total Backward Packets,0
Total Length of Fwd Packets,0
...,...
Idle Mean,0
Idle Std,0
Idle Max,0
Idle Min,0


In [ ]:
# Checking column names
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [ ]:
# Removing leading and trailing spaces from column names
df.columns = df.columns.str.strip()

In [ ]:
# Filtering the dataset based on the 'Label' column
benign_data = df[df['Label'] == 'Benign']
ddos_data = df[df['Label'] == 'DDoS']

# Printing the class distribution to ensure we have the correct splits
print("Class Distribution in Dataset:")
print(df['Label'].value_counts())


Class Distribution in Dataset:
Label
DDoS      128027
BENIGN     97718
Name: count, dtype: int64


In [ ]:
# Checking for missing values
missing_values = X.isnull().sum()

# Checking for infinite values
infinite_values = (X == float('inf')).sum() + (X == -float('inf')).sum()

print("Missing values in each column:\n", missing_values)
print("Total number of infinite values:", infinite_values)

Missing values in each column:
 Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
                              ..
Active Min                     0
Idle Mean                      0
Idle Std                       0
Idle Max                       0
Idle Min                       0
Length: 78, dtype: int64
Total number of infinite values: Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
                              ..
Active Min                     0
Idle Mean                      0
Idle Std                       0
Idle Max                       0
Idle Min                       0
Length: 78, dtype: int64


In [ ]:
# Checking for the maximum and minimum values of the features
print(X.describe())

       Destination Port  Flow Duration  Total Fwd Packets  \
count      225745.00000   2.257450e+05      225745.000000   
mean         8879.61946   1.624165e+07           4.874916   
std         19754.64740   3.152437e+07          15.422874   
min             0.00000  -1.000000e+00           1.000000   
25%            80.00000   7.118000e+04           2.000000   
50%            80.00000   1.452333e+06           3.000000   
75%            80.00000   8.805237e+06           5.000000   
max         65532.00000   1.199999e+08        1932.000000   

       Total Backward Packets  Total Length of Fwd Packets  \
count           225745.000000                225745.000000   
mean                 4.572775                   939.463346   
std                 21.755356                  3249.403484   
min                  0.000000                     0.000000   
25%                  1.000000                    26.000000   
50%                  4.000000                    30.000000   
75%             

In [ ]:
# Checking for infinity or extremely large values in X_train and X_test
import numpy as np

# Checking if there are any infinite values in the dataset
print("Any Infinite values in X_train: ", np.isinf(X_train).sum())
print("Any Infinite values in X_test: ", np.isinf(X_test).sum())

# Replacing infinite values with a large number or NaN (for later removal)
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Checking if there are any NaN values now
print("Any NaN values in X_train: ", X_train.isna().sum())
print("Any NaN values in X_test: ", X_test.isna().sum())

# Replacing NaN values with the median of each column (or another strategy)
X_train.fillna(X_train.median(), inplace=True)
X_test.fillna(X_test.median(), inplace=True)

# Scaling again
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Any Infinite values in X_train:  Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
                              ..
Active Min                     0
Idle Mean                      0
Idle Std                       0
Idle Max                       0
Idle Min                       0
Length: 78, dtype: int64
Any Infinite values in X_test:  Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
                              ..
Active Min                     0
Idle Mean                      0
Idle Std                       0
Idle Max                       0
Idle Min                       0
Length: 78, dtype: int64
Any NaN values in X_train:  Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0

In [ ]:
# Checking the maximum values in the dataset
print("Maximum values in X_train:\n", X_train.max())
print("Maximum values in X_test:\n", X_test.max())

Maximum values in X_train:
 Destination Port                   65532.0
Flow Duration                  119999872.0
Total Fwd Packets                   1932.0
Total Backward Packets              2942.0
Total Length of Fwd Packets       120783.0
                                  ...     
Active Min                      50100000.0
Idle Mean                      120000000.0
Idle Std                        65300000.0
Idle Max                       120000000.0
Idle Min                       120000000.0
Length: 78, dtype: float64
Maximum values in X_test:
 Destination Port                   65519.0
Flow Duration                  119999937.0
Total Fwd Packets                   1392.0
Total Backward Packets              2502.0
Total Length of Fwd Packets       183012.0
                                  ...     
Active Min                     100000000.0
Idle Mean                      120000000.0
Idle Std                        64300000.0
Idle Max                       120000000.0
Idle Min       

In [ ]:
# Hybrid model core

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, GRU, Dense, Dropout

model = Sequential()

# CNN Part
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(sequence_length, num_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten before passing to GRU
model.add(Flatten())

# GRU Part
model.add(GRU(64, return_sequences=True))
model.add(GRU(32))

# Dense Layers for final output
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Capping the values at the 99th percentile of each feature
max_vals = X_train.quantile(0.99, axis=0)

# Clipping the values in both X_train and X_test column-wise
X_train = X_train.apply(lambda col: col.clip(upper=max_vals[col.name]), axis=0)
X_test = X_test.apply(lambda col: col.clip(upper=max_vals[col.name]), axis=0)

# Scaling the data
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Adding CNN layers

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Adding GRU layer

model.add(GRU(units=64, return_sequences=False))
model.add(Dropout(0.3))

# Dense layers for final prediction

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))  # 'softmax' for multi-class classification

In [ ]:
# Step 4: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9999557022303927
Classification Report:
               precision    recall  f1-score   support

      BENIGN       1.00      1.00      1.00     19544
        DDoS       1.00      1.00      1.00     25605

    accuracy                           1.00     45149
   macro avg       1.00      1.00      1.00     45149
weighted avg       1.00      1.00      1.00     45149

